In [ ]:
# This calculates where a lens should go in GPI to re-image the Lyot stop
# in the new PupilCam

# Created 2021 Dec 6 by E.S

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product

%matplotlib qt

In [2]:
# Try grids of

# n_l: 1.2 to 3.0
# R: 10 mm to 1000 mm

n_l_array = [1.45,2.47] # np.linspace(1.2, 3.0, num=50, endpoint=True)
R1_array = np.linspace(1., 500., num=200, endpoint=True)
R1_array = np.append(R1_array,[1e8]) # to approximate at inf
R2_array = -np.linspace(1., 500., num=200, endpoint=True)
R2_array = np.append(R2_array,[11e8]) # to approximate at inf
d_l_array = np.linspace(0.5, 5., num=200, endpoint=True)
#M_array = np.linspace(-0.384,-0.9, num=200, endpoint=True)

In [3]:
# Thick lensmaker's formula: 1/f =

def thick_lens(n_l_pass, R1_pass, R2_pass, d_l_pass):
    '''
    returns focal length, using 
    
    1/f = A * [B + C]
    '''
    
    A = np.subtract(n_l_pass,1.)
    
    B = np.subtract(1./R1_pass,1./R2_pass)
    
    C = np.divide(np.multiply(A,d_l_pass),np.multiply(d_l_pass,np.multiply(R1_pass,R2_pass)))
    
    f_inv = np.multiply(A,np.add(B,C))

    return 1./f_inv

In [4]:
test = thick_lens(1.5, 30, 50, 0.1)
print(test)

146.34146341463415


In [5]:
# make DataFrame holding permutations of lens parameters

df = pd.DataFrame(product(n_l_array, R1_array, R2_array, d_l_array), 
                  columns=["n_l","R1","R2","d_l"])

In [6]:
# find effective focal length for each combination

df["f"] = thick_lens(n_l_pass=df["n_l"],
                     R1_pass=df["R1"],
                     R2_pass=df["R2"],
                     d_l_pass=df["d_l"])

In [7]:
# apply magnification constraint to fit image on detector: M_mag = -0.384
# s_i = f*(1-M)

#M_mag = -0.384
M_mag = -0.384
df["s_i"] = np.multiply(df["f"],np.subtract(1.,M_mag))
df["s_o"] = np.multiply(df["f"],np.divide(M_mag-1,M_mag))

# sanity check
df["f_test"] = np.divide(1.,np.add(np.divide(1.,df["s_i"]),np.divide(1.,df["s_o"]))) 

In [8]:
plt.hist(df["R1"], bins=200)
plt.show()

In [9]:
# apply cut such that lens position would only move a few mm from current

'''
df_physical1 = df.where(np.logical_and(df["s_o"] > 106.7 + 21.3 - 2,
                                      df["s_o"] < 106.7 + 21.3 + 2)).dropna()

'''
df_physical1 = df.copy()

In [10]:
len(df_physical1)

16160400

In [11]:
# apply cut such that focal length is < 65 mm

df_physical2 = df_physical1.where(df_physical1["f"] < 65).dropna()

In [12]:
len(df_physical2)

4998200

In [13]:
# apply cut such that si+so is close to (106.7+137.35) mm

df_physical3 = df_physical2.where(np.logical_and(
                                        np.add(df_physical2["s_i"],df_physical2["s_o"]) > 106.7+137.35-2,
                                        np.add(df_physical2["s_i"],df_physical2["s_o"]) < 106.7+137.35+2)).dropna()

In [14]:
len(df_physical3)

74400

In [15]:
# add a column to show residuals of si+so with needed
df_physical3["siso_resid"] = np.subtract(np.add(df_physical3["s_i"],df_physical3["s_o"]),106.7+137.35)
# add column to show residuals of movement from current
df_physical3["movement_resid"] = np.subtract(df_physical3["s_o"], 106.7 + 21.3)

In [16]:
df_physical3["siso_sum"] = np.add(df_physical3["s_i"],df_physical3["s_o"])

In [29]:
plt.hist(df_physical3["siso_sum"],bins=100)
plt.show()

In [31]:
# where are both residuals small?

#cond_siso = np.abs(df_physical3["siso_resid"]) < 2. # redundant
cond_movement = np.abs(df_physical3["movement_resid"]) < 30.
#cond_movement = True
#df_physical4 = df_physical3.where(np.logical_and(cond_movement,cond_siso)).dropna()
df_physical4 = df_physical3.where(cond_movement).dropna()

In [33]:
# where are both residuals small?

#cond_siso = np.abs(df_physical3["siso_resid"]) < 2. # redundant
cond_compact = np.subtract(np.add(df_physical3["s_o"],df_physical3["f"]),106.7) < 65.65
#cond_movement = True
#df_physical4 = df_physical3.where(np.logical_and(cond_movement,cond_siso)).dropna()
df_physical5 = df_physical4.where(cond_compact).dropna()

In [35]:
len(df_physical4)

74400

In [26]:
len(df_physical4)

74400

In [30]:
plt.hist(df_physical4["f"], bins=200)
plt.show()

In [36]:
plt.hist(df_physical4["R2"], bins=200)
plt.show()